In [1]:
#threadpool
import threadpool
import nltk

In [2]:
import threadpool
import time
import urllib2
 
urls = [
    'http://www.amazon.com', 
    'http://www.ebay.com', 
    'http://www.alibaba.com' 
]
 
def myRequest(url):
    resp = urllib2.urlopen(url)
    print url, resp.getcode()
 
 
def timeCost(request, n):
  print "Elapsed time: %s" % (time.time()-start)
 
start = time.time()
pool = threadpool.ThreadPool(5)
reqs = threadpool.makeRequests(myRequest, urls, timeCost)
[ pool.putRequest(req) for req in reqs ]
pool.wait()
dir(pool)
help(pool.wait)

http://www.alibaba.com 200
Elapsed time: 0.0956239700317
http://www.amazon.com 200
Elapsed time: 0.70273900032
http://www.ebay.com 200
Elapsed time: 0.75469493866
Help on method wait in module threadpool:

wait(self) method of threadpool.ThreadPool instance
    Wait for results, blocking until all have arrived.



In [3]:
lines = [
    'A thread pool is an object that maintains a pool of worker threads to perform time consuming operations in parallel. It assigns jobs to the threads by putting them in a work request queue, where they are picked up by the next available thread. ',
    'This then performs the requested operation in the background and puts the results in another queue. i want to eat an apple'
]

def cut(line):
    return [[word.lower() for word in nltk.word_tokenize(sentence) ]for sentence in nltk.sent_tokenize(line)]


for line in lines:
    print cut(line)

[['a', 'thread', 'pool', 'is', 'an', 'object', 'that', 'maintains', 'a', 'pool', 'of', 'worker', 'threads', 'to', 'perform', 'time', 'consuming', 'operations', 'in', 'parallel', '.'], ['it', 'assigns', 'jobs', 'to', 'the', 'threads', 'by', 'putting', 'them', 'in', 'a', 'work', 'request', 'queue', ',', 'where', 'they', 'are', 'picked', 'up', 'by', 'the', 'next', 'available', 'thread', '.']]
[['this', 'then', 'performs', 'the', 'requested', 'operation', 'in', 'the', 'background', 'and', 'puts', 'the', 'results', 'in', 'another', 'queue', '.'], ['i', 'want', 'to', 'eat', 'an', 'apple']]


In [4]:
words_sentences = [[]]
def cut_with_thread(line, words_sentences):
    _t = [[word.lower() for word in nltk.word_tokenize(sentence) ]for sentence in nltk.sent_tokenize(line)]
    print "_t : ", _t
    words_sentences.extend(_t)
    
_args = []
_args.append({'line':lines, 'words_sentences': words_sentences})
words_reqs = threadpool.makeRequests(cut_with_thread, 
                                     args_list = _args, 
                                     callback = timeCost)
[pool.putRequest(req) for req in words_reqs]
pool.wait()
print words_sentences


[[]]


Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/threadpool.py", line 158, in run
    result = request.callable(*request.args, **request.kwds)
TypeError: cut_with_thread() takes exactly 2 arguments (1 given)


In [5]:
fpath = '/data/opinion_spam/real/reviewsNew.txt'
#fpath = '/data/opinion_spam/real/10000lines'
f = open(fpath)
ofpath = '/data/opinion_spam/real/reviewsNew.txt_out'
#ofpath = '/data/opinion_spam/real/10000lines_out'


reviews = []

def cut_reviews_to_lines(line):
    _tmps = line.split('\t')
    if len(_tmps) == 8:
        if _tmps[3] > 1:
            reviews.append(_tmps[7])

for line in f:
    cut_reviews_to_lines(line)

print 'reviews list size : ', len(reviews)


    

reviews list size :  5803516


In [6]:
import string
import re
def cut_reviews(review):
    i = 0
    try:
        sentences = nltk.sent_tokenize(review)
        _lists = []
        for sentence in sentences:
#             _list = [word.lower() for word in nltk.word_tokenize(sentence)]
            _list = [word.lower() for word in sentence.split()]
            _list = [re.sub(r'[.?!\'",)(;&%:]', '', word) for word in _list  ]
            _lists.append(_list)
        return _lists
    except:
        i += 1

def cut_reviews_thread(review):
    _list = [[word.lower() for word in nltk.word_tokenize(sentence)] for sentence in nltk.sent_tokenize(review)]



In [7]:
%timeit cut_reviews('i want to eat an apple. what about you?')

1000 loops, best of 3: 303 µs per loop


In [8]:
#切割评论文件成为按词分割的句子
import time
t0 = time.time()
of = open(ofpath, 'a', 1024 * 1024 * 10)
for line in reviews:
    _lists = cut_reviews(line)
    if type(_lists) == list:
        for _list in _lists:
            of.write(" ".join(_list)+ "\n")
of.close()
t1 = time.time()

print 'over', str(t1-t0)

over 5950.58209395


In [9]:
# t0 = time.time()
# bigpool = threadpool.ThreadPool(num_workers = 32)
# bigreqs = threadpool.makeRequests(cut_reviews, reviews)
# [bigpool.putRequest(req) for req in bigreqs]
# bigpool.wait
# print 'over'
# t1 = time.time()

# print 'over', str(t1-t0)

In [10]:
5838923 / (9851/15.02) / 3600

2.472975768393508